In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [58]:
# Load datasets
df1 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/original/csv/TrafficLabelling/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', encoding='latin1')
df2 = pd.read_csv('/home/wahba/Documents/cicid/CICFlowMeter/data/daily/2025-12-02_Flow.csv', encoding='latin1')
df3 = pd.read_csv('/home/wahba/Documents/nids5/csv/flows_2025-12-04_9.csv', encoding='latin1')
df4 = pd.read_csv('/home/wahba/Documents/nids5_2/csv/flows_2025-12-02_1.csv', encoding='latin1')

/tmp/ipykernel_72584/1129530535.py:2: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/original/csv/TrafficLabelling/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', encoding='latin1')


## Dataset cleaning and  column standardizations

In [59]:
# Cleaning column names
col_names = {col: col.strip() for col in df1.columns}
df1.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df2.columns}
df2.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df3.columns}
df3.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df4.columns}
df4.rename(columns=col_names, inplace=True)

del col_names

In [60]:
print(df1.columns.tolist())
print(df2.columns.tolist())
print(df3.columns.tolist())
print(df4.columns.tolist())

['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH 

In [61]:
# Filter out only SQLi attacks from df1
df1_sqli = df1[df1['Label'] == 'Web Attack  Sql Injection'].copy()

# Standardize Protocol representation    
df1_sqli['Protocol'] = df1_sqli['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})
df2['Protocol'] = df2['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})
df3['Protocol'] = df3['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})
df4['Protocol'] = df4['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})

# Generate flow_ids
df1_sqli['flow_key'] = (
    df1_sqli['Source IP'].astype(str) + ':' +
    df1_sqli['Source Port'].astype(int).astype(str) + '-' +
    df1_sqli['Destination IP'].astype(str) + ':' +
    df1_sqli['Destination Port'].astype(int).astype(str) + '-' +
    df1_sqli['Protocol'].astype(str)
    )
df2['flow_key'] = (
    df2['Src IP'].astype(str) + ':' +
    df2['Src Port'].astype(str) + '-' +
    df2['Dst IP'].astype(str) + ':' +
    df2['Dst Port'].astype(str) + '-' +
    df2['Protocol'].astype(str)
    )
df3['flow_key'] = (
    df3['Source IP'].astype(str) + ':' +
    df3['Source Port'].astype(str) + '-' +
    df3['Destination IP'].astype(str) + ':' +
    df3['Destination Port'].astype(str) + '-' +
    df3['Protocol'].astype(str)
    )
df4['flow_key'] = (
    df4['Source IP'].astype(str) + ':' +
    df4['Source Port'].astype(str) + '-' +
    df4['Destination IP'].astype(str) + ':' +
    df4['Destination Port'].astype(str) + '-' +
    df4['Protocol'].astype(str)
    )

# Standardize column names across datasets following df1 naming
column_mapping = {
    'flow_key': 'flow_key',
    'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Total Backward Packets': 'Total Backward Packets',
    'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
    'Total Length of Bwd Packets': 'Total Length of Bwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Min Packet Length': 'Min Packet Length',
    'Max Packet Length': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Average Packet Size',
    'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
    'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
    'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward': 'Init_Win_bytes_backward',
    'act_data_pkt_fwd': 'act_data_pkt_fwd',
    'min_seg_size_forward': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
}

column_mapping_df2 = {
    'flow_key': 'flow_key',
    'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Bwd packets': 'Total Backward Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Total Length of Bwd Packet': 'Total Length of Bwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'CWR Flag Count': 'CWE Flag Count',  # Note: CWR vs CWE
    'ECE Flag Count': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Average Packet Size',
    'Fwd Segment Size Avg': 'Avg Fwd Segment Size',
    'Bwd Segment Size Avg': 'Avg Bwd Segment Size',
    'Fwd Bytes/Bulk Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Packet/Bulk Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Bulk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Bytes/Bulk Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Packet/Bulk Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Bulk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
}

df1_sqli = df1_sqli[list(column_mapping.keys())].rename(columns=column_mapping)
df2 = df2[list(column_mapping_df2.keys())].rename(columns=column_mapping_df2)
df3 = df3[list(column_mapping.keys())].rename(columns=column_mapping)
df4 = df4[list(column_mapping.keys())].rename(columns=column_mapping)

## Flow comparison

In [62]:
# Filter datasets to only include flows present in df1_sqli

sqli_keys = df1_sqli['flow_key'].unique().tolist()
df2_sqli = df2[df2['flow_key'].isin(sqli_keys)].copy()
df3_sqli = df3[df3['flow_key'].isin(sqli_keys)].copy()
df4_sqli = df4[df4['flow_key'].isin(sqli_keys)].copy()

print(df1_sqli.shape)
print(df2_sqli.shape)
print(df3_sqli.shape)
print(df4_sqli.shape)

print(df1_sqli['flow_key'].value_counts())
print(df2_sqli['flow_key'].value_counts())
print(df3_sqli['flow_key'].value_counts())
print(df4_sqli['flow_key'].value_counts())

(21, 78)
(9, 78)
(9, 78)
(9, 78)
flow_key
172.16.0.1:36196-192.168.10.50:80-TCP    2
172.16.0.1:36198-192.168.10.50:80-TCP    2
172.16.0.1:36200-192.168.10.50:80-TCP    2
172.16.0.1:36202-192.168.10.50:80-TCP    2
172.16.0.1:36204-192.168.10.50:80-TCP    2
172.16.0.1:36206-192.168.10.50:80-TCP    2
172.16.0.1:36208-192.168.10.50:80-TCP    2
172.16.0.1:36242-192.168.10.50:80-TCP    2
172.16.0.1:36244-192.168.10.50:80-TCP    2
172.16.0.1:36210-192.168.10.50:80-TCP    1
172.16.0.1:36212-192.168.10.50:80-TCP    1
172.16.0.1:36214-192.168.10.50:80-TCP    1
Name: count, dtype: int64
flow_key
172.16.0.1:36196-192.168.10.50:80-TCP    1
172.16.0.1:36198-192.168.10.50:80-TCP    1
172.16.0.1:36200-192.168.10.50:80-TCP    1
172.16.0.1:36202-192.168.10.50:80-TCP    1
172.16.0.1:36204-192.168.10.50:80-TCP    1
172.16.0.1:36206-192.168.10.50:80-TCP    1
172.16.0.1:36208-192.168.10.50:80-TCP    1
172.16.0.1:36210-192.168.10.50:80-TCP    1
172.16.0.1:36212-192.168.10.50:80-TCP    1
Name: count, dtype: 

In [63]:
# List of flow keys that appear in all datasets
flow_list = [
    "172.16.0.1:36196-192.168.10.50:80-TCP",
    "172.16.0.1:36198-192.168.10.50:80-TCP", 
    "172.16.0.1:36200-192.168.10.50:80-TCP",
    "172.16.0.1:36202-192.168.10.50:80-TCP",
    "172.16.0.1:36204-192.168.10.50:80-TCP",
    "172.16.0.1:36206-192.168.10.50:80-TCP",
    "172.16.0.1:36208-192.168.10.50:80-TCP",
    "172.16.0.1:36210-192.168.10.50:80-TCP",
    "172.16.0.1:36212-192.168.10.50:80-TCP"
]

In [64]:
flow_df1 = df1_sqli[df1_sqli['flow_key'] == flow_list[0]].iloc[[0]] # keep only the first row
flow_df2 = df2_sqli[df2_sqli['flow_key'] == flow_list[0]]
flow_df3 = df3_sqli[df3_sqli['flow_key'] == flow_list[0]]
flow_df4 = df4_sqli[df4_sqli['flow_key'] == flow_list[0]]

print(f'''
      {flow_df1.shape}
      {flow_df2.shape}
      {flow_df3.shape}
      {flow_df4.shape}
''')


      (1, 78)
      (1, 78)
      (1, 78)
      (1, 78)



In [65]:
# Flow features comparison 
flow_df1 = df1_sqli[df1_sqli['flow_key'] == flow_list[0]].iloc[[0]] # keep only the first row
flow_df2 = df2_sqli[df2_sqli['flow_key'] == flow_list[0]]
flow_df3 = df3_sqli[df3_sqli['flow_key'] == flow_list[0]]
flow_df4 = df4_sqli[df4_sqli['flow_key'] == flow_list[0]]

print(f'''
      {flow_df1.shape}
      {flow_df2.shape}
      {flow_df3.shape}
      {flow_df4.shape}
''')


# Flow Duration
print(f'''
    FLOW DURATION
    {flow_df1['Flow Duration']}
    {flow_df2['Flow Duration']}
    {flow_df3['Flow Duration']}
    {flow_df4['Flow Duration']}
    ''')

# Timestamp
print(f'''
    TIMESTAMP
    {flow_df1['Timestamp']}
    {flow_df2['Timestamp']}
    {flow_df3['Timestamp']}
    {flow_df4['Timestamp']}
    ''')

# 
print(f'''
    TOTAL FORWARD PACKETS
    {flow_df1['Total Fwd Packets']}
    {flow_df2['Total Fwd Packets']}
    {flow_df3['Total Fwd Packets']}
    {flow_df4['Total Fwd Packets']}
    ''')

print(f'''
    TOTAL BACKWARD PACKETS
    {flow_df1['Total Backward Packets']}
    {flow_df2['Total Backward Packets']}
    {flow_df3['Total Backward Packets']}
    {flow_df4['Total Backward Packets']}
    ''')

print(f'''
    TOTAL FIN FLAGS
    {flow_df1['FIN Flag Count']}
    {flow_df2['FIN Flag Count']}
    {flow_df3['FIN Flag Count']}
    {flow_df4['FIN Flag Count']}
    ''')

print(f'''
    TOTAL RST FLAGS
    {flow_df1['RST Flag Count']}
    {flow_df2['RST Flag Count']}
    {flow_df3['RST Flag Count']}
    {flow_df4['RST Flag Count']}
    ''')


      (1, 78)
      (1, 78)
      (1, 78)
      (1, 78)


    FLOW DURATION
    90293    5006127.0
Name: Flow Duration, dtype: float64
    47    5022024
Name: Flow Duration, dtype: int64
    106    5.013535e+06
Name: Flow Duration, dtype: float64
    98    5.022832
Name: Flow Duration, dtype: float64
    

    TIMESTAMP
    90293    6/7/2017 10:40
Name: Timestamp, dtype: object
    47    02/12/2025 04:52:14 AM
Name: Timestamp, dtype: object
    106    2025-12-04 15:33:07
Name: Timestamp, dtype: object
    98    2025-12-02 04:52:14
Name: Timestamp, dtype: object
    

    TOTAL FORWARD PACKETS
    90293    4.0
Name: Total Fwd Packets, dtype: float64
    47    4
Name: Total Fwd Packets, dtype: int64
    106    7
Name: Total Fwd Packets, dtype: int64
    98    7
Name: Total Fwd Packets, dtype: int64
    

    TOTAL BACKWARD PACKETS
    90293    4.0
Name: Total Backward Packets, dtype: float64
    47    4
Name: Total Backward Packets, dtype: int64
    106    5
Name: Total Backward Packets

In [66]:
# Flow features comparison 
flow_df1 = df1_sqli[df1_sqli['flow_key'] == flow_list[1]] # keep only the first row
flow_df2 = df2_sqli[df2_sqli['flow_key'] == flow_list[1]]
flow_df3 = df3_sqli[df3_sqli['flow_key'] == flow_list[1]]
flow_df4 = df4_sqli[df4_sqli['flow_key'] == flow_list[1]]

print(f'''
      {flow_df1.shape}
      {flow_df2.shape}
      {flow_df3.shape}
      {flow_df4.shape}
''')


# Flow Duration
print(f'''
    FLOW DURATION
    {flow_df1['Flow Duration']}
    {flow_df2['Flow Duration']}
    {flow_df3['Flow Duration']}
    {flow_df4['Flow Duration']}
    ''')

print(f'''
    Flow IAT Max
    {flow_df1['Flow IAT Max']}
    {flow_df2['Flow IAT Max']}
    {flow_df3['Flow IAT Max']}
    {flow_df4['Flow IAT Max']}
    ''')

print(f'''
    Flow IAT Min
    {flow_df1['Flow IAT Min']}
    {flow_df2['Flow IAT Min']}
    {flow_df3['Flow IAT Min']}
    {flow_df4['Flow IAT Min']}
    ''')

print(f'''
    Flow IAT Std
    {flow_df1['Flow IAT Std']}
    {flow_df2['Flow IAT Std']}
    {flow_df3['Flow IAT Std']}
    {flow_df4['Flow IAT Std']}
    ''')

print(f'''
    Fwd IAT Total
    {flow_df1['Fwd IAT Total']}
    {flow_df2['Fwd IAT Total']}
    {flow_df3['Fwd IAT Total']}
    {flow_df4['Fwd IAT Total']}
    ''')

print(f'''
    Fwd IAT Max
    {flow_df1['Fwd IAT Max']}
    {flow_df2['Fwd IAT Max']}
    {flow_df3['Fwd IAT Max']}
    {flow_df4['Fwd IAT Max']}
    ''')

print(f'''
    Fwd IAT Mean
    {flow_df1['Fwd IAT Mean']}
    {flow_df2['Fwd IAT Mean']}
    {flow_df3['Fwd IAT Mean']}
    {flow_df4['Fwd IAT Mean']}
    ''')

print(f'''
    Fwd IAT Std
    {flow_df1['Fwd IAT Std']}
    {flow_df2['Fwd IAT Std']}
    {flow_df3['Fwd IAT Std']}
    {flow_df4['Fwd IAT Std']}
    ''')

print(f'''
    Bwd IAT Total
    {flow_df1['Bwd IAT Total']}
    {flow_df2['Bwd IAT Total']}
    {flow_df3['Bwd IAT Total']}
    {flow_df4['Bwd IAT Total']}
    ''')

print(f'''
    Bwd IAT Max
    {flow_df1['Bwd IAT Max']}
    {flow_df2['Bwd IAT Max']}
    {flow_df3['Bwd IAT Max']}
    {flow_df4['Bwd IAT Max']}
    ''')

print(f'''
    Bwd IAT Mean
    {flow_df1['Bwd IAT Mean']}
    {flow_df2['Bwd IAT Mean']}
    {flow_df3['Bwd IAT Mean']}
    {flow_df4['Bwd IAT Mean']}
    ''')

print(f'''
    Bwd IAT Std
    {flow_df1['Bwd IAT Std']}
    {flow_df2['Bwd IAT Std']}
    {flow_df3['Bwd IAT Std']}
    {flow_df4['Bwd IAT Std']}
    ''')

print(f'''
    Bwd IAT Std
    {flow_df1['Bwd IAT Std']}
    {flow_df2['Bwd IAT Std']}
    {flow_df3['Bwd IAT Std']}
    {flow_df4['Bwd IAT Std']}
    ''')





      (2, 78)
      (1, 78)
      (1, 78)
      (1, 78)


    FLOW DURATION
    90329    5069927.0
90351         42.0
Name: Flow Duration, dtype: float64
    54    5075894
Name: Flow Duration, dtype: int64
    119    5.072581e+06
Name: Flow Duration, dtype: float64
    113    5.076494
Name: Flow Duration, dtype: float64
    

    Flow IAT Max
    90329    5000920.0
90351         42.0
Name: Flow IAT Max, dtype: float64
    54    5006833.0
Name: Flow IAT Max, dtype: float64
    119    5.002951e+06
Name: Flow IAT Max, dtype: float64
    113    5.006833
Name: Flow IAT Max, dtype: float64
    

    Flow IAT Min
    90329     4.0
90351    42.0
Name: Flow IAT Min, dtype: float64
    54    9.0
Name: Flow IAT Min, dtype: float64
    119    0.0
Name: Flow IAT Min, dtype: float64
    113    0.0
Name: Flow IAT Min, dtype: float64
    

    Flow IAT Std
    90329    1885988.001
90351          0.000
Name: Flow IAT Std, dtype: float64
    54    2.038573e+06
Name: Flow IAT Std, dtype: float64
    119

In [67]:
print(df3['Active Max'].value_counts())

Active Max
0.000000e+00    1776
2.418447e+04       1
1.815673e+06       1
1.831331e+06       1
1.981236e+06       1
                ... 
2.572775e+04       1
2.578282e+04       1
2.565789e+04       1
2.488995e+05       1
2.077868e+05       1
Name: count, Length: 261, dtype: int64


- CICIDS2017 use microseconds